In [1]:
import requests
import pandas as pd
import numpy as np
import random
pd.options.mode.chained_assignment = None  # default='warn' # Removes Settingwithcopywarning

# Getting FPL API keys by converting them to json
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

# Create DataFrames from data. Elements = players
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

# Get databases for each position
player_names_df = elements_df[['first_name','second_name','element_type','team', 'total_points', 'now_cost']]
player_names_df['position'] = player_names_df.element_type.map(elements_types_df.set_index('id').singular_name)
player_names_df['team'] = player_names_df.team.map(teams_df.set_index('id').name)

gk_names_df = player_names_df.loc[player_names_df.position =='Goalkeeper']
def_names_df = player_names_df.loc[player_names_df.position =='Defender']
mid_names_df = player_names_df.loc[player_names_df.position =='Midfielder']
for_names_df = player_names_df.loc[player_names_df.position =='Forward']

# Randomly generate team by position
team_gk = gk_names_df.sample(n=2)
team_def = def_names_df.sample(n=5)
team_mid = mid_names_df.sample(n=5)
team_for = for_names_df.sample(n=3)
team_df = pd.concat([team_gk, team_def, team_mid, team_for])

while team_df['now_cost'].sum() >= 1000:
    team_gk = gk_names_df.sample(n=2)
    team_def = def_names_df.sample(n=5)
    team_mid = mid_names_df.sample(n=5)
    team_for = for_names_df.sample(n=3)
    team_df = pd.concat([team_gk, team_def, team_mid, team_for])

print(team_df['now_cost'].sum())

team_df


835


,first_name,second_name,element_type,team,total_points,now_cost,position
408,Pierluigi,Gollini,1,Spurs,0,45,Goalkeeper
124,Bailey,Peacock-Farrell,1,Burnley,3,45,Goalkeeper
406,Joe,Rodon,2,Spurs,27,45,Defender
47,Frédéric,Guilbert,2,Aston Villa,0,45,Defender
269,Ben,Davies,2,Liverpool,0,40,Defender
181,Michael,Keane,2,Everton,127,50,Defender
174,Joachim,Andersen,2,Crystal Palace,93,45,Defender
391,Heung-Min,Son,3,Spurs,228,100,Midfielder
52,Jacob,Ramsey,3,Aston Villa,28,45,Midfielder
257,Takumi,Minamino,3,Liverpool,48,55,Midfielder


In [12]:
# Randomly select player to transfer out using randint
player_out = team_df.iloc[random.randint(0,14)]

# Remove selected player from team. Need to find out why only inplace=False works
team_df = team_df.drop(player_out.name,inplace=False)

# Create mask to filter out possible players to transfer in
mask = (player_names_df.position == player_out.position) & (player_names_df.now_cost <= player_out.now_cost)

# Apply mask to player database
player_in_df = player_names_df[mask]
# Randomly select from df of filtered players
pop_players = len(player_in_df.index) - 1
player_in = player_in_df.iloc[[random.randint(0, pop_players)]]

# Add selected player to team
team_df = pd.concat([team_df, player_in])
# Sort team based on position for neatness
team_df.sort_values(by=['element_type'])

team_df

NameError: name 'pd' is not defined

In [3]:
# Transpose player_out data from column to row. Could not rename as 'player_out' because it caused errors after repeated runs
player_out_x = pd.DataFrame([player_out.transpose()])
# Make transfer df by concat two rows
transfers = pd.concat([player_out_x, player_in])
# Reset index to make loc for column renaming consistent
transfers.reset_index(inplace=True)
# Rename 'index' column to 'actions' column
transfers.iloc[0,0]= 'Transfer out'
transfers.iloc[1,0]= 'Transfer in'
transfers = transfers.rename(columns={'index':'Action'})
# Hide index for neatness
transfers.style.hide_index()


NameError: name 'pd' is not defined